<h3><b>Importation des librairies</b></h1>

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup

In [2]:
url = "https://books.toscrape.com/catalogue/category/books_1/index.html"

reponse = requests.get(url=url)

reponse.status_code

200

In [3]:
def get_book_themes(url:str="https://books.toscrape.com/catalogue/category/books_1/index.html"):
    reponse = requests.get(url=url)
    if reponse.status_code == 200:
        data_str = reponse.content
        data_html = soup(data_str, "html.parser")
        data = [theme.get_text().strip() for theme in data_html.find_all("div", attrs={"class":"side_categories"})[0].find_all("li")[1:]]
        
        return data
    return None


In [4]:
def get_book_link_themes(url:str="https://books.toscrape.com/catalogue/category/books_1/index.html"):
    reponse = requests.get(url=url)
    if reponse.status_code == 200:
        data_str = reponse.content
        data_html = soup(data_str, "html.parser")
        data = ["https://books.toscrape.com/catalogue/category" + link["href"].split("..")[1] for link in data_html.find_all("div", attrs={"class":"side_categories"})[0].find_all("a")[1:]]
        return data
    return None
    

In [ ]:
# table categorie
themes = get_book_themes()
links = get_book_link_themes()

df_categorie = pd.DataFrame({"name":themes, "link":links})
df_categorie.head()

,name,lien
0,Travel,https://books.toscrape.com/catalogue/category/...
1,Mystery,https://books.toscrape.com/catalogue/category/...
2,Historical Fiction,https://books.toscrape.com/catalogue/category/...
3,Sequential Art,https://books.toscrape.com/catalogue/category/...
4,Classics,https://books.toscrape.com/catalogue/category/...


In [6]:
if __name__ == "__main__":
    url = "https://books.toscrape.com/catalogue/category/books_1/index.html"
    data = get_book_link_themes(url)[:2]
    display(data)

['https://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html']

Pour chaque lien, je vais aller recuperer les liens de chaque livre qui me permettront de recupérer la photo de livre, son titre, son titre, la quantité qu'il y a en stock, sa référence (UPC) et sa desciption.

<h3><b>Table livre</b></h3>

In [7]:
links = get_book_link_themes()
links[:2]

['https://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html']

In [8]:
url = links[0]

reponse = requests.get(url=url)
reponse.status_code

200

In [9]:
data_str = reponse.content
data_html = soup(data_str, "html.parser")
data_html


<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    Travel | 
     Books to Scrape - Sandbox

</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="
    
" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../../../../static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="../../../../static/oscar/css/styles.css" rel="stylesheet"

In [10]:
data_html.find_all("article", attrs={"class":"product_pod"})[0]

<article class="product_pod">
<div class="image_container">
<a href="../../../its-only-the-himalayas_981/index.html"><img alt="It's Only the Himalayas" class="thumbnail" src="../../../../media/cache/27/a5/27a53d0bb95bdd88288eaf66c9230d7e.jpg"/></a>
</div>
<p class="star-rating Two">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="../../../its-only-the-himalayas_981/index.html" title="It's Only the Himalayas">It's Only the Himalayas</a></h3>
<div class="product_price">
<p class="price_color">£45.17</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [11]:
get_book_link_themes()[0]

'https://books.toscrape.com/catalogue/category/books/travel_2/index.html'

In [12]:
# On recupere tous les liens des livres pour un thème données
def get_links_books_for_one_theme(url: str):
    reponse = requests.get(url)
    if reponse.status_code == 200:
        data_str = reponse.content
        data_html = soup(data_str, "html.parser")
        links = data_html.find_all("div", attrs={"class":"image_container"})
        data = [link.find("a")["href"].split("../../")[1] for link in links]
        data_clean = ["https://books.toscrape.com/catalogue/" + x.split("../")[1] for x in data]
        return data_clean
                

# On recupère tous les liens pour tous les thèmes
def get_links_books_for_all_theme():
    links_themes = get_book_link_themes()
    data = []
    for link in links_themes:
        links = get_links_books_for_one_theme(url=link)
        data += links
    return data

links_book = get_links_books_for_all_theme()
links_book[:5]

['https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html',
 'https://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html',
 'https://books.toscrape.com/catalogue/see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html',
 'https://books.toscrape.com/catalogue/vagabonding-an-uncommon-guide-to-the-art-of-long-term-world-travel_552/index.html',
 'https://books.toscrape.com/catalogue/under-the-tuscan-sun_504/index.html']

Pour chaque lien, nous allons maintenant recupérer les informations pour notre table book

In [86]:
def get_info_one_book(url:str=links_book[0]):
    reponse = requests.get(url)
    if reponse.status_code == 200:
        data_str = reponse.content
        data_html = soup(data_str, "html.parser")
        
        titre = data_html.find("div", attrs={"class":"col-sm-6 product_main"}).find("h1").get_text()
        price = data_html.find("div", attrs={"class":"col-sm-6 product_main"}).find("p").get_text()[1:]
        disponibilite = data_html.find("div", attrs={"class":"col-sm-6 product_main"}).find("p").find_next("p").get_text().strip().split("(")[1]
        nb_livre_disponible = disponibilite.split(" ")[0]
        nb_starts = data_html.find("p", attrs={"class":"star-rating Two"}).attrs["class"][1]
        image = "https://books.toscrape.com/" + data_html.find("div", attrs={"class":"item active"}).find("img")["src"].split("../../")[1]
        description = data_html.find("div", attrs={"id":"product_description"}).find_next("p").get_text()
        upc = data_html.find("table", attrs={"class":"table table-striped"}).find("td").get_text()
        
        data = {"name": titre, "price":price, "description":description, 
                "reference":upc, "availability":nb_livre_disponible, "note":nb_starts, "image":image}
        
    return data

# get_info_one_book()

df = get_info_one_book()

pd.DataFrame([df])

# https://books.toscrape.com/
    

,name,price,description,reference,availability,note,image
0,It's Only the Himalayas,45.17,"“Wherever you go, whatever you do, just . . . ...",a22124811bfa8350,19,Two,https://books.toscrape.com/media/cache/6d/41/6...


In [ ]:
# data1 = {"name": "a", "price":15, "description":"imagine ...", 
#         "reference":"986sdfg6", "availability":2, "note":5, "image":"www.book..."}

# data2 = {"name": "b", "price":45, "description":"imagine ...", 
#         "reference":"9ersd5g6", "availability":19, "note":2, "image":"www.book..."}

# df1 = pd.DataFrame([data1, data2])
# df1


,name,price,description,reference,availability,note,image
0,a,15,imagine ...,986sdfg6,2,5,www.book...
1,b,45,imagine ...,9ersd5g6,19,2,www.book...
